In [1]:
import numpy as np
import pandas as pd
import scipy.stats


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('./data/provData/x_train.csv').set_index('Provider')
#data_test = pd.read_csv('./data/provData/x_test_inout.csv').set_index('Provider')

In [3]:
data.drop(columns = ['Unnamed: 0','Unnamed: 0.1'], inplace=True)

In [4]:
data.isna().sum()

Age                            0
Gender                         0
BeneID                         0
ClaimID                        0
NumDiag                        0
OPAnnualReimbursementAmt       0
IPAnnualReimbursementAmt       0
TotalClaim                     0
InscCovPercent                12
DailyCharge                    0
DupRecord                      0
WhetherDead                    0
Alzheimer                      0
HeartFailure                   0
KidneyDisease                  0
Cancer                         0
ObstrPulmonary                 0
Depression                     0
Diabetes                       0
IschemicHeart                  0
Osteoporasis                   0
RheumatoidArthritis            0
Stroke                         0
LargeClaims                    0
AdmissionDays_in               0
ClaimDays_in                   0
ClaimDays_out                  0
NumChronics_in                 0
NumChronics_out                0
NumProc_in                     0
NumProc_ou

In [5]:
data.fillna(0, inplace=True)

In [6]:
# data['LargeClaims'] = data['ClaimID'] > 1300

# data['LargeClaims'].value_counts()

# #data.columns

# smallclaims = data[data.LargeClaims == 0]
# largeclaims = data[data.LargeClaims == 1]


In [12]:
x_unscaled.columns

Index(['BeneID', 'ClaimID', 'NumDiag', 'OPAnnualReimbursementAmt',
       'IPAnnualReimbursementAmt', 'TotalClaim', 'InscCovPercent',
       'DailyCharge', 'DupRecord', 'WhetherDead', 'KidneyDisease',
       'IschemicHeart', 'AdmissionDays_in', 'ClaimDays_in', 'ClaimDays_out',
       'NumChronics_in', 'NumChronics_out', 'NumProc_in', 'NumProc_out',
       'State_out', 'NumProc_Range', 'ClaimDays_Range', 'TotalClaim_Range',
       'InscCovPercent_Range', 'DailyCharge_Range', 'docDegMax', 'docEignMean',
       'patDegMax', 'patMANN'],
      dtype='object')

In [13]:
y = data['PotentialFraud']
x_unscaled = data.drop(columns = ['Age','State_in','Gender','AdmissionDays_in','docBtwnMean','patEignMean',
                                  'Alzheimer', 'HeartFailure','Cancer', 'ObstrPulmonary', 
                                  'Depression', 'Diabetes','Osteoporasis','patBtwnMean',
                                  'RheumatoidArthritis', 'Stroke','docMANN','LargeClaims',
                                  'PotentialFraud'])

In [14]:
from sklearn import preprocessing
x = preprocessing.scale(x_unscaled)

In [15]:
from sklearn.model_selection import train_test_split as tts

In [16]:
x_train, x_test, y_train, y_test = tts(x,y,test_size=1.0/3, random_state=0)

print('Original: {}, {}'.format(x.shape, y.shape))
print('Training: {}, {}'.format(x_train.shape, y_train.shape))
print('Test:     {}, {}'.format(x_test.shape, y_test.shape))

Original: (5410, 28), (5410,)
Training: (3606, 28), (3606,)
Test:     (1804, 28), (1804,)


In [30]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix,classification_report


def ClassMetrics(x,y, model):
    logit_tr_acc = model.score(x, y)
    logit_tr_pr, logit_tr_re, logit_tr_f1, _ = precision_recall_fscore_support(y, logit.predict(x))

    print(" Logit Train Accuracy : %1.3f" % (logit_tr_acc))
    print(" Logit Train Precision: %1.3f (no fraud) and %1.3f (fraud)" % (logit_tr_pr[0], logit_tr_pr[1]))
    print(" Logit Train Recall   : %1.3f (no fraud) and %1.3f (fraud)" % (logit_tr_re[0], logit_tr_re[1]))
    print(" Logit Train F1 Score : %1.3f (no fraud) and %1.3f (fraud)" % (logit_tr_f1[0], logit_tr_f1[1]))
    
    print(confusion_matrix(y,model.predict(x)))
    #print(classification_report(y,model.predict(x)))
    
    y_probs_logit = pd.DataFrame(model.predict_proba(x))[1]
    fpr, tpr, thresholds = roc_curve(y, y_probs_logit)
    auc = roc_auc_score(y, y_probs_logit)  # Computes auc
    
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw,
            label='ROC logit (area = %0.2f)' % auc)

    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0, 1.02])
    plt.ylim([0, 1.02])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [17]:
%matplotlib inline
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.__version__

'2.0.0'

In [18]:
[0]*13

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
x_train.shape

(3606, 28)

In [24]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=[None,28])
Y = tf.placeholder(dtype=tf.float32, shape=[None,1])
W = tf.Variable(initial_value=tf.random_uniform(shape=[28, 2]), dtype=tf.float32)
b = tf.Variable(initial_value = [0,0], dtype=tf.float32)
z = tf.nn.softmax(tf.matmul(X,W)+b)

loss = tf.reduce_mean(-Y*tf.log(z))
opt = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = opt.minimize(loss)


In [32]:
num_iterations = 30000
init = tf.global_variables_initializer()
feed_dict = {X:x_train, Y:y_train[:,np.newaxis]}
with tf.Session() as sess:
    init.run()
    for i in range(num_iterations):
        sess.run(train_op, feed_dict = feed_dict)
    print('slope coefficients:')
    print(sess.run(W))
    print('intercept term:')
    print(sess.run(b))
    prob = sess.run(z, feed_dict=feed_dict)
    myTarget = np.argmax(prob, axis=1)
    print('my prediction:')
    print(myTarget)

slope coefficients:
[[0.41811034 0.3830196 ]
 [0.7864691  0.80813915]
 [0.61021686 0.7648383 ]
 [0.53559387 0.5500573 ]
 [0.39319783 0.4151778 ]
 [0.65331316 0.6556074 ]
 [0.2914189  0.34058025]
 [0.8089391  0.6738984 ]
 [0.44612488 0.4465655 ]
 [0.4115275  0.4133639 ]
 [0.6530273  0.5562443 ]
 [0.7544298  0.7771717 ]
 [0.42977956 0.2778    ]
 [0.64344656 0.6207447 ]
 [0.6199111  0.6679561 ]
 [0.6345769  0.64262384]
 [0.6880743  0.8091498 ]
 [0.40227324 0.40313777]
 [0.522757   0.52921855]
 [0.42186573 0.36253944]
 [0.3683273  0.4066942 ]
 [0.3360954  0.333374  ]
 [0.6033198  0.5801901 ]
 [0.94675773 0.96996206]
 [0.83530146 0.83282214]
 [0.71734744 0.71484333]
 [0.3396541  0.34153897]
 [0.21585368 0.22019447]]
intercept term:
[-0.02666036  0.02666046]
my prediction:
[1 1 1 ... 1 0 0]


In [33]:
confusion_matrix(myTarget,y_train)

array([[ 834,  146],
       [2435,  191]])

In [37]:
logit_tr_pr, logit_tr_re, logit_tr_f1, _ = precision_recall_fscore_support(y_train, myTarget)

print(" Logit Train Precision: %1.3f (no fraud) and %1.3f (fraud)" % (logit_tr_pr[0], logit_tr_pr[1]))
print(" Logit Train Recall   : %1.3f (no fraud) and %1.3f (fraud)" % (logit_tr_re[0], logit_tr_re[1]))
print(" Logit Train F1 Score : %1.3f (no fraud) and %1.3f (fraud)" % (logit_tr_f1[0], logit_tr_f1[1]))

 Logit Train Precision: 0.851 (no fraud) and 0.073 (fraud)
 Logit Train Recall   : 0.255 (no fraud) and 0.567 (fraud)
 Logit Train F1 Score : 0.393 (no fraud) and 0.129 (fraud)


Horrible Performance (no hidden layers)

In [40]:
tf.reset_default_graph()
with tf.variable_scope('multinomial_model', reuse=tf.AUTO_REUSE) as linear_model:
    # Setup the placeholder to input features and labels
    X = tf.placeholder(shape=[None,28], dtype=tf.float32, name='X')
    Y = tf.placeholder(shape=[None,1], dtype=tf.float32, name='Y')
    
    # define prediction using the dense layer and then define our loss
    y = tf.layers.dense(X,units=2, activation=tf.nn.softmax, name='multinomial')
    loss = tf.reduce_mean(-Y*tf.log(y))
    
    # specify the optimizer
    opt = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    
    # pull out the coefficients from the dense layer
    weights = tf.get_variable('multinomial/weights', shape=[28,1])
    bias = tf.get_variable('multinomial/bias', shape=[2,])
    
    # define the training op, and compute the confusion matrix
    train_op = opt.minimize(loss) # define the training operation
    tgt = tf.argmax(y,axis=1) # convert probabilites to integer label
    cm = tf.confusion_matrix(tgt, y_train)
    

In [41]:
num_iterations = 10000
init = tf.global_variables_initializer()
feed_dict = {X:x_train, Y:y_train[:,np.newaxis]}
with tf.Session() as sess:
    init.run()
    for i in range(num_iterations):
        sess.run(train_op, feed_dict=feed_dict)
    print("The weights are:")        
    print(sess.run(weights))
    print("\nThe biases are:")
    print(sess.run(bias))
    tgt2 = np.argmax(sess.run(y,feed_dict=feed_dict), axis=1)
    print('\nCounting Predicted Classes')
    print(pd.Series(tgt2).value_counts())
    print('\nConfusion matrix:')
    print(sess.run(cm, feed_dict=feed_dict))

The weights are:
[[-0.2405416 ]
 [ 0.23900354]
 [ 0.08518237]
 [-0.05285555]
 [-0.32173648]
 [-0.25893193]
 [-0.36629674]
 [ 0.42501968]
 [ 0.34311426]
 [-0.1772885 ]
 [-0.14850467]
 [-0.06506535]
 [-0.04815006]
 [-0.4219667 ]
 [ 0.3438874 ]
 [-0.3124784 ]
 [ 0.12044287]
 [-0.12241527]
 [-0.32314837]
 [ 0.35406947]
 [ 0.3932485 ]
 [ 0.07562655]
 [ 0.00398391]
 [-0.43596426]
 [ 0.2920916 ]
 [-0.1482651 ]
 [ 0.35183537]
 [ 0.37251854]]

The biases are:
[ 0.00727385 -0.00727373]

Counting Predicted Classes
0    2156
1    1450
dtype: int64

Confusion matrix:
[[1996  160]
 [1273  177]]


## Classification: Keras

In [2]:
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl

import sklearn

In [3]:
mpl.rcParams['figure.figsize'] = (12,10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
